# Quickstart for Carsus

In this guide you will learn how to create atomic data files suitable for [TARDIS](https://github.com/tardis-sn/tardis).

<div class="alert alert-info">

**Note:**

Get familiar with the [Notation in Carsus](development/notation.rst) and learn how to correctly select ions.
    
</div>

## Atomic Weights and Ionization Energies (NIST)

Download atomic weights and ionization energies from the National Institute of Standards and Technology (NIST).

In [1]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets


In [2]:
atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-Zn')

[ carsus.io.nist.weightscomp][   INFO] - Downloading data from the carsus-dat-nist repository (weightscomp.py:72)


[                py.warnings][WARNING] - /home/runner/micromamba/envs/carsus/lib/python3.7/site-packages/urllib3/connectionpool.py:1110: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  InsecureRequestWarning,
 (warnings.py:110)


[  carsus.io.nist.ionization][   INFO] - Downloading ionization energies from the carsus-data-nist repo. (ionization.py:89)


[                py.warnings][WARNING] - /home/runner/micromamba/envs/carsus/lib/python3.7/site-packages/urllib3/connectionpool.py:1110: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  InsecureRequestWarning,
 (warnings.py:110)


## Levels, Lines, Collisions & Cross-sections

Carsus supports three sources of energy levels and spectral lines: **GFALL**, **CHIANTI** and **CMFGEN**.

### GFALL

The Robert Kurucz's Atomic Linelist (GFALL) reader is the main source of **levels and lines**.

<div class="alert alert-warning">

**Warning:**
    
Creating a `GFALLReader` instance is **required**.

</div>

In [3]:
!wget -qO /tmp/gfall.dat https://media.githubusercontent.com/media/tardis-sn/carsus-db/master/gfall/gfall_latest.dat

In [4]:
from carsus.io.kurucz import GFALLReader

gfall_reader = GFALLReader('H-Zn',
                           '/tmp/gfall.dat')

[     carsus.io.kurucz.gfall][WARNING] - A specific combination to identify unique levels from GFALL data has not been given. Defaulting to ["energy", "j"]. (gfall.py:129)


### CHIANTI

The Chianti Atomic Database reader provides levels and lines but also **collision strengths**.

<div class="alert alert-info">

**Note:**

Creating a `ChiantiReader` instance is **optional**. 

</div>

In [5]:
from carsus.io.chianti_ import ChiantiReader

chianti_reader = ChiantiReader('H-He', 
                               collisions=True, 
                               priority=20)

By default `priority` parameter is set to `10`. Increase this value if you want to keep CHIANTI levels and lines over GFALL.

### CMFGEN

The atomic database of [CMFGEN](http://kookaburra.phyast.pitt.edu/hillier/web/CMFGEN.htm) is a source of levels, lines and (optionally) **ionization energies**, **photoionization cross-sections** and **collisions**.

<div class="alert alert-info">
    
**Note:**

Creating a `CMFGENReader` instance is **optional**. 

</div>

<div class="alert alert-warning">

**Warning:**
    
Cross-sections require data from `H 0`, use this the reader with enough `priority` to select levels from this ion.

</div>

In [6]:
from carsus.io.cmfgen import CMFGENReader

cmfgen_reader = CMFGENReader.from_config('Si 0-1', 
                                         '/tmp/atomic', 
                                         priority=30,
                                         ionization_energies=True,
                                         cross_sections=True,
                                         collisions=False,
                                         temperature_grid=None,
                                         drop_mismatched_labels=True)

[      carsus.io.cmfgen.base][WARNING] - Selecting H 0 from CMFGEN (required to ingest cross-sections). (base.py:557)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for H 0. (base.py:571)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for Si 0. (base.py:571)


[      carsus.io.cmfgen.base][   INFO] - Configuration schema found for Si 1. (base.py:571)


[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for H 0. (base.py:854)


[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for Si 0. (base.py:854)


[      carsus.io.cmfgen.base][   INFO] - Loading atomic data for Si 1. (base.py:854)


## Zeta Data

Long & Knigge's ground state recombinations fractions ($\zeta$).

In [7]:
from carsus.io.zeta import KnoxLongZeta

zeta_data = KnoxLongZeta()

## Nuclear decay data

Carsus also supports the decay radiation data of all the nuclides at the [NNDC Archives](https://www.nndc.bnl.gov/ensdfarchivals/). The ENSDF data is stored in CSV format in the repository [carsus-data-nndc](https://github.com/tardis-sn/carsus-data-nndc).

### NNDC

The NNDCReader instance looks for the `carsus-data-nndc` repository in the local system at the path specified by the argument `dirname`. If the data is to be downloaded from the Github repository directly , the `remote` argument should be set to `True`.

<div class="alert alert-info">
    
**Note:**

Creating a `NNDCReader` instance is **optional**. 

</div>

In [8]:
from carsus.io.nuclear import NNDCReader

nndc_reader = NNDCReader(remote=True)

[     carsus.io.nuclear.nndc][WARNING] - Failed to clone the repository.
Check if the repository already exists at {DECAY_DATA_SOURCE_DIR} (nndc.py:44)


[     carsus.io.nuclear.nndc][   INFO] - Parsing decay data from: /home/runner/Downloads/carsus-data-nndc/csv (nndc.py:50)


fatal: destination path '/home/runner/Downloads/carsus-data-nndc' already exists and is not an empty directory.


## Create an Atomic Data File

Finally, create a `TARDISAtomData` object and dump the data with the `to_hdf` method.

In [9]:
from carsus.io.output import TARDISAtomData

atom_data = TARDISAtomData(atomic_weights,
                           ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           cmfgen_reader,
                           nndc_reader)

[      carsus.io.output.base][   INFO] - Ingesting energy levels. (base.py:365)


[     carsus.io.kurucz.gfall][   INFO] - Parsing GFALL from: /tmp/gfall.dat (gfall.py:182)


[      carsus.io.output.base][   INFO] - GFALL selected species: Li 0, Li 1, Be 0, Be 1, Be 2, B 0, B 1, B 2, B 3, C 0, C 1, C 2, C 3, N 0, N 1, N 2, N 3, N 4, N 5, O 0, O 1, O 2, O 3, O 4, O 5, F 0, F 1, F 2, F 3, F 4, F 5, Ne 0, Ne 1, Ne 2, Ne 3, Ne 4, Ne 5, Na 0, Na 1, Na 2, Na 3, Na 4, Na 5, Mg 0, Mg 1, Mg 2, Mg 3, Mg 4, Mg 5, Al 0, Al 1, Al 2, Al 3, Al 4, Al 5, Si 2, Si 3, Si 4, Si 5, P 0, P 1, P 2, P 3, P 4, P 5, S 0, S 1, S 2, S 3, S 4, S 5, Cl 0, Cl 1, Cl 2, Cl 3, Cl 4, Ar 0, Ar 1, Ar 2, Ar 3, Ar 4, K 0, K 1, K 2, K 3, K 4, Ca 0, Ca 1, Ca 2, Ca 3, Ca 4, Ca 5, Ca 6, Ca 7, Ca 8, Sc 0, Sc 1, Sc 2, Sc 3, Sc 4, Sc 5, Sc 6, Sc 7, Sc 8, Ti 0, Ti 1, Ti 2, Ti 3, Ti 4, Ti 5, Ti 6, Ti 7, Ti 8, V 0, V 1, V 2, V 3, V 4, V 5, V 6, V 7, V 8, Cr 0, Cr 1, Cr 2, Cr 3, Cr 4, Cr 5, Cr 6, Cr 7, Cr 8, Mn 0, Mn 1, Mn 2, Mn 3, Mn 4, Mn 5, Mn 6, Mn 7, Mn 8, Fe 0, Fe 1, Fe 2, Fe 3, Fe 4, Fe 5, Fe 6, Fe 7, Fe 8, Co 0, Co 1, Co 2, Co 3, Co 4, Co 5, Co 6, Co 7, Co 8, Ni 0, Ni 1, Ni 2, Ni 3, Ni 4, Ni 5, Ni 

[      carsus.io.output.base][   INFO] - Chianti selected species: He 0, He 1. (base.py:410)


[      carsus.io.output.base][   INFO] - CMFGEN selected species: H 0, Si 0, Si 1. (base.py:414)


...

[      carsus.io.output.base][   INFO] - Ingesting collisional strengths. (base.py:688)


[      carsus.io.output.base][   INFO] - Matching collisions and levels. (base.py:700)


[      carsus.io.output.base][   INFO] - Ingesting photoionization cross-sections. (base.py:806)


[      carsus.io.output.base][   INFO] - Matching levels and cross sections. (base.py:809)


[      carsus.io.output.base][   INFO] - Finished. (base.py:131)


In [10]:
atom_data.to_hdf('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5')

[                py.warnings][WARNING] - /home/runner/micromamba/envs/carsus/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['Element', 'Parent E(level)', 'Uncertainty', 'JPi', 'Decay Mode',
       'Q Value Uncertainty', 'T1/2 (txt)', 'Gammas Balance', 'X-Rays Balance',
       'B- Balance', 'B+ Balance', 'Conversion Electrons Balance',
       'Auger Electrons Balance', 'Neutrinos Balance', 'Recoil Balance',
       'Neutrons Balance', ' Protons Balance', 'Alphas Balance', 'Sum Balance',
       'Q-effective Balance', 'Missing Energy Balance', 'Ratio Balance',
       'Daughter', 'Radiation', 'Rad subtype', 'Uncertainty.1',
       'Uncertainty.3', 'Uncertainty.4'],
      dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)
 (warnings.py:110)


[                py.warnings][WARNING] - /home/runner/micromamba/envs/carsus/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->values] [items->None]

  exec(code_obj, self.user_global_ns, self.user_ns)
 (warnings.py:110)


[                py.warnings][WARNING] - /home/runner/micromamba/envs/carsus/lib/python3.7/site-packages/pyarrow/serialization.py:165: FutureWarning: The SparseDataFrame class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  if isinstance(obj, pd.SparseDataFrame):
 (warnings.py:110)


[                py.warnings][WARNING] - /home/runner/micromamba/envs/carsus/lib/python3.7/site-packages/pyarrow/serialization.py:176: FutureWarning: The SparseSeries class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  if isinstance(obj, pd.SparseSeries):
 (warnings.py:110)


[      carsus.io.output.base][   INFO] - Signing TARDISAtomData. (base.py:1334)


[      carsus.io.output.base][   INFO] - Format Version: 1.0 (base.py:1335)


[      carsus.io.output.base][   INFO] - MD5: 8f71b4735a2a69e2b14f89ec1aec1e8e (base.py:1336)


[      carsus.io.output.base][   INFO] - UUID1: a371f994615a11efad316045bdbf9970 (base.py:1337)


You are done! Now you can use your file to run TARDIS simulations.

### Metadata

Carsus stores metadata inside the HDF5 files to ensure reproducibility. This metadata includes a checksum for each stored table, version number or checksum of selected datasets, and versions of relevant packages. 

In [11]:
import pandas as pd

In [12]:
store = pd.HDFStore('kurucz_cd23_chianti_He_cmfgen_H_Si_I-II.h5', key='metadata')

In [13]:
store["metadata"]

value
field    key                                                    
format   version                                             1.0
md5sum   atom_data              634785f05c4868f929c85c02828d4ef8
         collisions_data        0f19bec97c3b770c6ed7e78be2cd1efd
         collisions_metadata    2139b8beb1db60823290c24aa0d5a464
         ionization_data        8e5cc288a013018b8f37eee48dfdb2c5
         levels_data            aa695d9348461d452e04970959f7ec32
         lines_data             aa58ecd7e85a89016d03e513df973995
         lines_metadata         736d8321848fa598443cbd807732a8dc
         macro_atom_data        1244eb0abb974e485ca387639c78d545
         macro_atom_references  4e6ae567911277dc6883a9a815b23163
         nuclear_decay_rad      51d08a42853fdbf2013e1b24e674a27d
         photoionization_data   b0c86b51740f1a4b620170b34daf0ae5
         zeta_data              f2496cec5c219dfb6d637df8e92fd3f0
datasets nist_weights                                        4.1
         nist_spectra                                       5.11
         gfall                  2704fbda0b8cba61bb70426234224464
         zeta                   a1d4bed2982e8d6a4f8b0076bf637e49
         chianti                                           9.0.1
         cmfgen                                       2016.11.15
software python                                            3.7.8
         carsus                              0.1.dev754+gbf6636a
         astropy                                           3.2.3
         numpy                                            1.15.4
         pandas                                            1.0.5
         pyarrow                                          0.14.1
         tables                                            3.6.1
         ChiantiPy                                         0.8.4

In [14]:
store.root._v_attrs

/._v_attrs (AttributeSet), 8 attributes:
   [CLASS := 'GROUP',
    DATE := '2024-08-23T14:18:58.891308+00:00',
    FORMAT_VERSION := '1.0',
    MD5 := '8f71b4735a2a69e2b14f89ec1aec1e8e',
    PYTABLES_FORMAT_VERSION := '2.1',
    TITLE := '',
    UUID1 := 'a371f994615a11efad316045bdbf9970',
    VERSION := '1.0']